In [1]:
! pip install accelerate -U
! pip install evaluate
! pip install sentence-transformers
! pip install SentencePiece
! pip install bert_score
! pip install --upgrade nltk
! pip install rouge_score

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 20.8 MB/s eta 0:00:00
  Attempting uninstall: nltk
    Found existing installation: nltk 3.2.4
    Uninstalling nltk-3.2.4:
      Successfully uninstalled nltk-3.2.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
preprocessing 0.1.13 requires nltk==3.2.4, but you have nltk 3.8.1 which is incompatible.
  Preparing metadata (setup.py) ... - done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=f405b8dcbb2ec076d1b5f43f77d62098da1f8e132edbb592e3ab0bfe7fc3e9e3
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532

In [2]:
import wandb
from transformers import DataCollatorForSeq2Seq
import pyarrow as pa
import pyarrow.dataset as ds
import pandas as pd
from datasets import Dataset

#  import T5
import torch
import torch.nn as nn
import torch.nn.functional as F
from transformers import T5Tokenizer, T5ForConditionalGeneration, T5Config
import numpy as np
import os

import torch
from datasets import load_dataset
from transformers import (
    RobertaTokenizerFast,
    RobertaForSequenceClassification,
    TrainingArguments,
    Trainer,
    DebertaTokenizer,
    DebertaForSequenceClassification,
    AutoConfig,
)

import logging
import evaluate
from evaluate import load
from datasets import load_dataset

from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer
from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer
from transformers import T5Tokenizer,T5ForConditionalGeneration

import nltk
from nltk.translate.bleu_score import corpus_bleu

import matplotlib.pyplot as plt

from  transformers  import  AutoTokenizer, AutoModelWithLMHead

import csv
from torch.utils.data import DataLoader, Dataset
import json
import pandas as pd
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
import pickle
import numpy as np
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report
from tqdm import tqdm


device=""
if torch.cuda.is_available():
    device = torch.device("cuda")
    print(f'Using GPU: {torch.cuda.get_device_name()}')
else:
    device = torch.device("cpu")
    print('Using CPU')

PATH="D:\\ghd\\NLP-Project\\webis-clickbait-22\\"
OUTPATH="D:\\ghd\\NLP-Project\\webis-clickbait-22\\output\\"
BATCH=2
T5_model = "t5-base"
sep='<s>'

2024-04-25 15:56:45.800582: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-25 15:56:45.800677: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-25 15:56:45.940538: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Using GPU: Tesla P100-PCIE-16GB


In [3]:
def fun(word1, word2, dp):

    # check if the result is already present in the dp table
    if dp[len(word2)][len(word1)] != -1:
        return dp[len(word2)][len(word1)]          

    # if both words are empty, then the number of operations required is 0
    if len(word1) == 0 and len(word2) == 0:
        return 0

    # if one of the words is empty, then the number of operations required is the length of the other word
    if len(word2) == 0:
        return len(word1)

    if len(word1) == 0:
        return len(word2)

    # add: insert a character in word1 to match word2
    add = fun(word1, word2[1:], dp) + 1

    # delete: delete a character in word1 to match word2
    delete = fun(word1[1:], word2, dp) + 1

    # replace: replace a character in word1 to match word2
    if word1[0] == word2[0]:
        # if the firsr character is same, then the number of operations will not increase
        replace = fun(word1[1:], word2[1:], dp)
    else:
        # else, the number of operations will increase
        replace = fun(word1[1:], word2[1:], dp) + 1

    # take the minimum of the three recursive calls and store in the dp table
    dp[len(word2)][len(word1)] = min(min(add, delete), replace)

    # return the result
    return dp[len(word2)][len(word1)]


def minDistance(word1: str, word2: str) -> int:
    # create the dp table with -1 as initial values
    dp = [[-1 for i in range(len(word1)+1)] for j in range(len(word2)+1)]

    # call the recursive function and return the result
    return fun(word1, word2, dp)


In [4]:
data=pd.read_json("/kaggle/input/webis-clickbait-22/test.jsonl",lines=True,encoding='utf-8')

In [5]:
def get_split_data(data,if_train):
    global label_encoder

    # y = data["spoiler"]
    Y=[]
    # for i in y:
    #     Y.append(" ".join(i))
    y=0
    new_data=data[['postText','targetParagraphs','targetTitle','targetDescription','targetKeywords','spoiler','tags']]
    dic1=new_data.to_dict(orient='records')
    tags=data["tags"]
    dic=[]
    for i in dic1:
        # print(i)
#         if(t not in i['tags']):
#             del i['postText']
#             del i['targetParagraphs']
#             del i['targetTitle']
#             del i['targetDescription']
#             del i['targetKeywords']
#             del i['tags']
#             del i['spoiler']
#             del i
#             continue

        i['postText']=i['postText'][0]
        targetparah=" ".join(i['targetParagraphs'])
        i['targetParagraphs']=targetparah
        j={}
        j['question']="question : "+i['postText']+"?"
        j['context']="context : "+i['targetParagraphs']
        dic.append(j)
        b=" ".join(i['spoiler'])
        if(b==" "):
            y+=1
        Y.append(" ".join(i['spoiler']))

        del i['postText']
        del i['targetParagraphs']
        del i['targetTitle']
        del i['targetDescription']
        del i['targetKeywords']
        del i['tags']
        del i['spoiler']
        # break
    # print("desccount: ",desccount)
    # print("keycount: ",keycount)
    # print(dic)
    X_q = [i['question'] for i in dic]
    X_q=np.array(X_q)
    X_c = [i['context'] for i in dic]
    X_c=np.array(X_c)
    print(y)
#     return X_q[:50],X_c[:50],Y[:50],tags[:50]
    return X_q,X_c,Y,tags

# x_test_q,x_test_c,y_test = get_split_data(data,False,"multi")
# test_df = pd.DataFrame(list(zip(x_test_q, x_test_c, y_test)), columns =['question', 'context', 'spoiler'])


In [6]:
from tqdm import tqdm

In [7]:
# decoded=[[],[],[],[]]
# spoiler=[]

In [8]:

x_test_q,x_test_c,y_test,tags = get_split_data(data,False)
test_df = pd.DataFrame(list(zip(x_test_q, x_test_c, y_test,tags)), columns =['question', 'context', 'spoiler','tags'])

0


In [9]:


M1="/kaggle/input/t5-dataset-proj/T5/checkpoint-3200"
M2="/kaggle/input/t5-passage/T5_overall"
M3="/kaggle/input/t5-multi/T5_overall"
M4="/kaggle/input/t5-phrase/T5_overall"
M5="/kaggle/input/t5-proj-1/T5_overall"
M6="/kaggle/input/t5-qa/T5_overall"

tokenizer1 = AutoTokenizer.from_pretrained(M1)
model1 = AutoModelWithLMHead.from_pretrained(M1)

tokenizer2 = AutoTokenizer.from_pretrained(M2)
model2 = AutoModelWithLMHead.from_pretrained(M2)

tokenizer3 = AutoTokenizer.from_pretrained(M3)
model3 = AutoModelWithLMHead.from_pretrained(M3)

tokenizer4 = AutoTokenizer.from_pretrained(M4)
model4 = AutoModelWithLMHead.from_pretrained(M4)

tokenizer5 = AutoTokenizer.from_pretrained(M5)
model5 = AutoModelWithLMHead.from_pretrained(M5)

tokenizer6 = AutoTokenizer.from_pretrained(M6)
model6 = AutoModelWithLMHead.from_pretrained(M6)

tokenizer5 = AutoTokenizer.from_pretrained(M5)

model1.to(device)
model2.to(device)
model3.to(device)
model4.to(device)
model5.to(device)
model6.to(device)

model_toke={0:[model1,tokenizer1],1:[model5,tokenizer5],2:[model6,tokenizer6],3:[]}
models={0:M1,1:M5,2:M6,3:"Mixed"}


/opt/conda/lib/python3.10/site-packages/transformers/models/auto/modeling_auto.py:1682: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  warnings.warn(
You set `add_prefix_space`. The tokenizer needs to be converted from the slow tokenizers


In [10]:
spoiler=test_df['spoiler'].tolist()

In [11]:
decoded=[]
for i in range(len(model_toke)):
    cur_decoded=[]
    for j in tqdm(range(len(test_df))):
        row=test_df.iloc[j]
        index=j

        if(i==3):
            tag=row['tags']
            if('phrase' in tag):
                model=model4
                tokenizer=tokenizer4
            elif('passage' in tag):
                model=model2
                tokenizer=tokenizer2
            elif('multi' in tag):
                model=model3
                tokenizer=tokenizer3
                
        else:
            model=model_toke[i][0].to(device)
            tokenizer=model_toke[i][1]
        
    # for index, row in tqdm(test_df.iterrows()):
#         row=test_df.iloc[j]
        # input_text = row['question'] + sep + row['context']
#         print(row)
        input_ids = tokenizer.encode(row['question'],row['context'], return_tensors="pt", max_length=512, truncation=True).to(device)
        outputs = model.generate(input_ids, max_length=512, early_stopping=True)
        cur_decoded.append(tokenizer.decode(outputs[0], skip_special_tokens=True))
#         decoded[i].append(cur_decoded)
        # if(index==10):
        #     break
    decoded.append(cur_decoded)

  0%|          | 0/1000 [00:00<?, ?it/s]/opt/conda/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:535: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(
100%|██████████| 1000/1000 [08:33<00:00,  1.95it/s]


In [12]:
# tag="phrase"
# x_test_q,x_test_c,y_test = get_split_data(data,False,tag)
# test_df = pd.DataFrame(list(zip(x_test_q, x_test_c, y_test)), columns =['question', 'context', 'spoiler'])

In [13]:
# M1="/kaggle/input/t5-dataset-proj/T5/checkpoint-3200"
# if (tag=='passage'):
#     M2="/kaggle/input/t5-passage/T5_overall"
# if(tag=='multi'):
#     M2="/kaggle/input/t5-multi/T5_overall"
# if(tag=='phrase'):
#     M2="/kaggle/input/t5-phrase/T5_overall"
# M3="/kaggle/input/t5-proj-1/T5_overall"
# M4="/kaggle/input/t5-qa/T5_overall"

# tokenizer1 = AutoTokenizer.from_pretrained(M1)
# model1 = AutoModelWithLMHead.from_pretrained(M1)

# tokenizer2 = AutoTokenizer.from_pretrained(M2)
# model2 = AutoModelWithLMHead.from_pretrained(M2)

# tokenizer3 = AutoTokenizer.from_pretrained(M3)
# model3 = AutoModelWithLMHead.from_pretrained(M3)

# tokenizer4 = AutoTokenizer.from_pretrained(M4)
# model4 = AutoModelWithLMHead.from_pretrained(M4)

# model_toke={0:[model1,tokenizer1],1:[model2,tokenizer2],2:[model3,tokenizer3],3:[model4,tokenizer4]}
# models={0:M1,1:M2,2:M3,3:M4}

# model1.to(device)
# model2.to(device)
# model3.to(device)
# model4.to(device)

In [14]:
# decoded2=[]
# for i in range(len(model_toke)):
#     model=model_toke[i][0].to(device)
#     tokenizer=model_toke[i][1]
#     cur_decoded=[]
#     for j in tqdm(range(len(test_df))):

#     # for index, row in tqdm(test_df.iterrows()):
#         row=test_df.iloc[j]
#         index=j
#         # input_text = row['question'] + sep + row['context']
#         input_ids = tokenizer.encode(row['question'],row['context'], return_tensors="pt", max_length=512, truncation=True).to(device)
#         outputs = model.generate(input_ids, max_length=512, num_beams=5, early_stopping=True)
#         cur_decoded.append(tokenizer.decode(outputs[0], skip_special_tokens=True))
#         decoded[i].append(cur_decoded)
        
#         # if(index==10):
#         #     break
#     decoded2.append(cur_decoded)

In [15]:
# spoiler2=test_df['spoiler'].tolist()


In [16]:
# tag="phrase"
# x_test_q,x_test_c,y_test = get_split_data(data,False,tag)
# test_df = pd.DataFrame(list(zip(x_test_q, x_test_c, y_test)), columns =['question', 'context', 'spoiler'])

In [17]:
# M1="/kaggle/input/t5-dataset-proj/T5/checkpoint-3200"
# if (tag=='passage'):
#     M2="/kaggle/input/t5-passage/T5_overall"
# if(tag=='multi'):
#     M2="/kaggle/input/t5-multi/T5_overall"
# if(tag=='phrase'):
#     M2="/kaggle/input/t5-phrase/T5_overall"
# M3="/kaggle/input/t5-proj-1/T5_overall"
# M4="/kaggle/input/t5-qa/T5_overall"

# tokenizer1 = AutoTokenizer.from_pretrained(M1)
# model1 = AutoModelWithLMHead.from_pretrained(M1)

# tokenizer2 = AutoTokenizer.from_pretrained(M2)
# model2 = AutoModelWithLMHead.from_pretrained(M2)

# tokenizer3 = AutoTokenizer.from_pretrained(M3)
# model3 = AutoModelWithLMHead.from_pretrained(M3)

# tokenizer4 = AutoTokenizer.from_pretrained(M4)
# model4 = AutoModelWithLMHead.from_pretrained(M4)

# model_toke={0:[model1,tokenizer1],1:[model2,tokenizer2],2:[model3,tokenizer3],3:[model4,tokenizer4]}
# models={0:M1,1:M2,2:M3,3:M4}

# model1.to(device)
# model2.to(device)
# model3.to(device)
# model4.to(device)

In [18]:
# decoded3=[]
# for i in range(len(model_toke)):
#     model=model_toke[i][0].to(device)
#     tokenizer=model_toke[i][1]
#     cur_decoded=[]
#     for j in tqdm(range(len(test_df))):

#     # for index, row in tqdm(test_df.iterrows()):
#         row=test_df.iloc[j]
#         index=j
#         # input_text = row['question'] + sep + row['context']
#         input_ids = tokenizer.encode(row['question'],row['context'], return_tensors="pt", max_length=512, truncation=True).to(device)
#         outputs = model.generate(input_ids, max_length=512, num_beams=5, early_stopping=True)
#         cur_decoded.append(tokenizer.decode(outputs[0], skip_special_tokens=True))
#         decoded[i].append(cur_decoded)
#         # if(index==10):
#         #     break
#     decoded3.append(cur_decoded)

In [19]:
# spoiler3=test_df['spoiler'].tolist()

In [20]:
# for i in spoiler1:
#     spoiler.append(i)
# for i in spoiler2:
#     spoiler.append(i)
# for i in spoiler3:
#     spoiler.append(i)

In [21]:
# models={0:M1,1:"Mix Models",2:M3,3:M4}


In [22]:
# decoded=[]
# spoiler=[]
# for i in tqdm(range(len(test_df))):
# # for i in range(len(test_df)):
#     # print(test_df.iloc[i])
        
#     q=test_df.iloc[i]['question']
#     c=test_df.iloc[i]['context']
#     enc1=tokenizer1(q,c,return_tensors='pt',padding=True,truncation=True,max_length=512)
#     out1=model1.generate(input_ids=enc1['input_ids'].to(device),attention_mask=enc1['attention_mask'].to(device),max_length=512,num_beams=4,early_stopping=True)
#     dec1=tokenizer1.decode(out1[0],skip_special_tokens=True)

#     enc2=tokenizer2(q,c,return_tensors='pt',padding=True,truncation=True,max_length=512)
#     out2=model2.generate(input_ids=enc2['input_ids'].to(device),attention_mask=enc2['attention_mask'].to(device),max_length=512,num_beams=4,early_stopping=True)
#     dec2=tokenizer2.decode(out2[0],skip_special_tokens=True)

#     enc3=tokenizer3(q,c,return_tensors='pt',padding=True,truncation=True,max_length=512)
#     out3=model3.generate(input_ids=enc3['input_ids'].to(device),attention_mask=enc3['attention_mask'].to(device),max_length=512,num_beams=4,early_stopping=True)
#     dec3=tokenizer3.decode(out3[0],skip_special_tokens=True)
#     penalty=200
#     edit1=0
#     if(len(dec1)<3):
#         edit1=penalty
#     decs=[dec1,dec2,dec3]
#     for j in range(1,3):
#         edit1+=minDistance(decs[0],decs[j])
#     edit2=0
#     if(len(dec2)<3):
#         edit2=penalty
#     for j in range(2):
#         edit2+=minDistance(decs[1],decs[j])
#     edit3=0
#     if(len(dec3)<3):
#         edit3=penalty
#     for j in range(2):
#         edit3+=minDistance(decs[2],decs[j])
    
#     if(edit1<=edit2 and edit1<=edit3):
#         dec=dec1
#     elif(edit2<=edit1 and edit2<=edit3):
#         dec=dec2
#     else:
#         dec=dec3
#     print(edit1,edit2,edit3)
#     print(decs)
#     decoded.append(dec)
#     spoiler.append(test_df.iloc[i]['spoiler'])
#     # break
#     if(i==10):
#         break
# decoded,spoiler

In [23]:
# print(decoded)
# print(spoiler)

In [24]:
# models
# # ensemble=[]
# for i in range(len(spoiler)):
#     predicted=[decoded[j][i] for j in range(len(decoded))]
#     edit=[]
#     for j in range(len(decoded)):
#         e=0
#         for k in range(len(decoded)):
#             e+=minDistance(decoded[j],predicted[k])
#         edit.append(e)
    
#     arg_min=np.argmin(edit)
#     ensemble.append(decoded[arg_min][i])



In [25]:
# decoded.append(ensemble)
# models[len(model_toke)]="Ensemble"

In [26]:
# blue
def calculate_bleu(bp,precisions):
    weights_1 = np.array([1,0,0,0])
    weights_2 = np.array([0.5,0.5,0,0])
    weights_3 = np.array([0.33,0.33,0.33,0])
    weights_4 = np.array([0.25,0.25,0.25,0.25])
    logp = np.log(precisions)
        
    res1 = bp*np.exp(np.dot(logp,weights_1))
    res2 = bp*np.exp(np.dot(logp,weights_2))
    res3 = bp*np.exp(np.dot(logp,weights_3))
    res4 = bp*np.exp(np.dot(logp,weights_4))
    
    return [res1,res2,res3,res4]

for i in range(len(decoded)):
    preds=decoded[i]
    refs=spoiler
    print(models[i])
    # print(len(preds),len(refs))
    bleu = evaluate.load("bleu")
    results = bleu.compute(predictions=preds, references=refs)
    print(calculate_bleu(results['brevity_penalty'],results['precisions']))
    print(results)  


/kaggle/input/t5-dataset-proj/T5/checkpoint-3200


[0.34600318245618045, 0.3104862387982203, 0.29293930343018165, 0.2761751311596719]
{'bleu': 0.2761751311596719, 'precisions': [0.4222290029750553, 0.3399950450078454, 0.30844531458574354, 0.29134751773049644], 'brevity_penalty': 0.8194680612137434, 'length_ratio': 0.8339589032381195, 'translation_length': 13109, 'reference_length': 15719}
/kaggle/input/t5-proj-1/T5_overall
[0.3613006226990809, 0.324869748885441, 0.30716835942796716, 0.2902232000980854]
{'bleu': 0.29022320009808544, 'precisions': [0.43491704374057316, 0.35163132137030995, 0.3207414565580251, 0.30368986454927605], 'brevity_penalty': 0.8307345685780844, 'length_ratio': 0.8435651122844965, 'translation_length': 13260, 'reference_length': 15719}
/kaggle/input/t5-qa/T5_overall
[0.29147603437762154, 0.24824472866613695, 0.22848066395371736, 0.21049424772822728]
{'bleu': 0.21049424772822725, 'precisions': [0.2947761194029851, 0.2138191818494328, 0.18723933562596035, 0.17401338564504962], 'brevity_penalty': 0.9888047748506654, 

In [27]:
! pip install rouge_score

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [28]:
# rouge
for i in range(len(decoded)):
    preds=decoded[i]
    refs=spoiler
    print(models[i])

    # print(len(preds),len(refs))
    rouge = evaluate.load("rouge")
    results = rouge.compute(predictions=preds, references=refs)
    print(results)

/kaggle/input/t5-dataset-proj/T5/checkpoint-3200


{'rouge1': 0.4943097878498586, 'rouge2': 0.36143800213585875, 'rougeL': 0.48815815122449413, 'rougeLsum': 0.4890123716176035}
/kaggle/input/t5-proj-1/T5_overall
{'rouge1': 0.499464380056328, 'rouge2': 0.36729743151214156, 'rougeL': 0.49339506538284184, 'rougeLsum': 0.49359611133534737}
/kaggle/input/t5-qa/T5_overall
{'rouge1': 0.3813434321729372, 'rouge2': 0.25989816837759583, 'rougeL': 0.3724993263683828, 'rougeLsum': 0.3727621129708058}
Mixed
{'rouge1': 0.5166575436154159, 'rouge2': 0.3769212515887963, 'rougeL': 0.5073376443773723, 'rougeLsum': 0.5080324420000566}


In [29]:
# bertscore
bert = evaluate.load("bertscore")

for i in range(len(decoded)):
    preds=decoded[i]
    refs=spoiler
    print(models[i])

    results = bert.compute(predictions=preds, references=refs,lang='en')
    precision = np.array(results['precision'])
    recall = np.array(results['recall'])
    f1 = np.array(results['f1'])

    print("Precision: ",precision.mean())
    print("Recall: ",recall.mean())
    print("F1: ",f1.mean()) 

/kaggle/input/t5-dataset-proj/T5/checkpoint-3200


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

...

model.safetensors:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Precision:  0.9050514265298844
Recall:  0.8979665455818177
F1:  0.9008519483804702
/kaggle/input/t5-proj-1/T5_overall
Precision:  0.9055770506858826
Recall:  0.8989530633687973
F1:  0.9016192344427109
/kaggle/input/t5-qa/T5_overall


Precision:  0.8833255555033683
Recall:  0.8800906920433045
F1:  0.8810096229910851
Mixed


Precision:  0.8951440616250038
Recall:  0.8908820875287056
F1:  0.8925490850806236


In [30]:
# meteor

meteor = evaluate.load("meteor")
for i in range(len(decoded)):
    preds=decoded[i]
    refs=spoiler
    print(models[i])

    results = meteor.compute(predictions=preds, references=refs)
    print(results)

[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /usr/share/nltk_data...


/kaggle/input/t5-dataset-proj/T5/checkpoint-3200
{'meteor': 0.43351262187061196}
/kaggle/input/t5-proj-1/T5_overall
{'meteor': 0.43932022271417104}
/kaggle/input/t5-qa/T5_overall
{'meteor': 0.3365142039750793}
Mixed
{'meteor': 0.4531464953234241}
